In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import BernoulliNB
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score

df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')


to_encode = []
for column in df.columns:
    if column == 'LoanID':
        continue
    if df[column].dtype == 'object':
        to_encode.append(column)
    elif df[column].dtype not in ['float64', 'int64', 'float', 'int']:
        to_encode.append(column)

for column in to_encode:
    ohe = LabelEncoder() #for now let's use labelEncoder
    # ohe = OneHotEncoder()
    print('column is ', column)
    df[column] = ohe.fit_transform(df[column])
    test_df[column] = ohe.transform(test_df[column])

print('encoding done')

bnb_clf = BernoulliNB(binarize=True)
train_df, validation_df = train_test_split(df, test_size=0.25, random_state=17)

train_df = train_df.drop(columns=['LoanID'])
validation_df = validation_df.drop(columns=['LoanID'])

x_train_df = train_df.drop(columns=['Default'])
y_train_df = train_df['Default']


bnb_clf.fit(x_train_df, y_train_df)

x_validation_df = validation_df.drop(columns=['Default'])
y_validation_df = validation_df['Default']

y_validation_pred = bnb_clf.predict(x_validation_df)

train_acc = accuracy_score(y_train_df, bnb_clf.predict(x_train_df))
valid_acc = accuracy_score(y_validation_df, y_validation_pred)

print(f'Training accuracy {train_acc}')
print(f'Validation accuracy {valid_acc}')


newdf = pd.DataFrame({"LoanID": test_df['LoanID'], "Default": bnb_clf.predict(X=test_df.drop(columns=['LoanID']))})
newdf.to_csv('bernoulli_naive_bayes_out.csv', index=False)

column is  Education
column is  EmploymentType
column is  MaritalStatus
column is  HasMortgage
column is  HasDependents
column is  LoanPurpose
column is  HasCoSigner
encoding done
Training accuracy 0.8833930564530341
Validation accuracy 0.8847072645388682
